<a href="https://colab.research.google.com/github/IseLei18-code/NBA-picks/blob/main/Sihaya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Introduction**

This following data analysis project aims to use past and recent data from NBA players and determine not only the bets on prizebets, more or less, and also the probability of us winning.

In [3]:
print("Hello World")

Hello World
